In [ ]:
import osmnx as ox
import networkx as nx
import geopandas as gpd
import pandas as pd
from tqdm.notebook import tqdm

### Assign depots
This notebook assigns depots to parcels per operator by selecting for each parcel the depot that is the closest in routed distance.

In [ ]:
# Manage inputs and outputs
graph_path = "../../results/network/accessible.graphml"
depots_path = "../../results/depots/per_operator/dhl.gpkg"
parcels_path = "../../results/parcels/per_operator/baseline_2022/dhl.gpkg"
output_path = "../../results/parcels/depots/baseline_2022/dhl.gpkg"

if "snakemake" in locals():
    graph_path = snakemake.input["graph"]
    depots_path = snakemake.input["depots"]
    parcels_path = snakemake.input["parcels"]
    output_path = snakemake.output[0]

In [ ]:
# Load network
graph = ox.load_graphml(graph_path)

In [ ]:
# Load depots
df_depots = gpd.read_file(depots_path)
df_depots = df_depots.rename(columns = { "node": "depot_node" })

In [ ]:
# Load parcels
df_parcels = gpd.read_file(parcels_path)
df_parcels = df_parcels.rename(columns = { "node": "parcel_node" })

In [ ]:
assert len(set(df_depots["operator"].unique())) == 1
assert len(set(df_parcels["operator"].unique())) == 1
assert df_depots["operator"].unique()[0] == df_parcels["operator"].unique()[0]

In [ ]:
# Extract individual nodes
depot_nodes = set(df_depots["depot_node"].unique())
parcel_nodes = set(df_parcels["parcel_node"].unique())

In [ ]:
# Calculate distances (coudl be parallelized))
df_distances = { "depot_node": [], "parcel_node": [], "distance": [] }
edge_lengths = nx.get_edge_attributes(graph, "length")

for depot_node in tqdm(depot_nodes):
    routes = nx.single_source_dijkstra_path(graph, depot_node, weight = "length")

    for parcel_node in parcel_nodes:
        route = routes[parcel_node]
    
        df_distances["depot_node"].append(depot_node)
        df_distances["parcel_node"].append(parcel_node)

        df_distances["distance"].append(sum([
            edge_lengths[(u, v, 0)]
            for u, v in zip(route[0:-1], route[1:])
        ]))

df_distances = pd.DataFrame(df_distances)

In [ ]:
df_distances = df_distances.sort_values(by = "distance")
df_distances = df_distances.drop_duplicates("parcel_node", keep = "first")
df_distances = df_distances.drop_duplicates(["parcel_node", "depot_node"], keep = "first")

In [ ]:
# Merge, sort, and drop duplicates to find closest assignments
df_assignment = pd.merge(df_parcels[["parcel_id", "parcel_node"]], df_distances[[
    "parcel_node", "depot_node"]], on = "parcel_node")

df_assignment = pd.merge(df_assignment, df_depots[[
    "depot_id", "depot_node"]], on = "depot_node")

df_assignment = df_assignment.sort_values(by = ["parcel_id", "depot_id"])
df_assignment = df_assignment.drop_duplicates("parcel_id", keep = "first")

In [ ]:
assert len(df_assignment) == len(df_parcels)
assert set(df_assignment["parcel_id"].unique()) == set(df_parcels["parcel_id"].unique())
assert len(df_assignment) == len(df_assignment["parcel_id"].unique())

In [ ]:
df_parcels = df_parcels.rename(columns = { "parcel_node": "node" })

df_parcels = pd.merge(df_parcels, df_assignment[[
    "parcel_id", "depot_id"]], on = "parcel_id", validate = "one_to_one")

In [ ]:
# Output
df_parcels.to_file(output_path)